In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
# 删除空数据
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [5]:
## 创建dataset
from torch.utils.data import Dataset

class MyDataSet(Dataset):
  def __init__(self) -> None:
    super().__init__()
    self.data = pd.read_csv("/content/drive/MyDrive/ChnSentiCorp_htl_all.csv")
    self.data = self.data.dropna()

  def __getitem__(self, index):
    return self.data.iloc[index]["review"], self.data.iloc[index]["label"] # data.iloc[index]取DataFrame中第index行的数据(从0开始)

  def __len__(self):
    return len(self.data)

In [6]:
dataset = MyDataSet()
for i in range(5):
  print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', np.int64(1))
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', np.int64(1))
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', np.int64(1))
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', np.int64(1))
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', np.int64(1))


In [7]:
# 划分数据集
from torch.utils.data import random_split
trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('温州人开的酒店，就是这副样子。总的来说还好，就是档次上不去。顶楼的餐厅不错，台塑牛扒很好。', np.int64(1))
('酒店比较一般，隔音不是很好，比较令人不满的是早餐，口味及品种皆不理想，希望能够改进。', np.int64(0))
('服务尚可,携程网协议价偏高.本人发现其他公司协议价只有366元(含早).', np.int64(1))
('香港马可最吸引人的地方当然是她便利的条件啦；附近的美心酒楼早茶很不错（就在文化中心里头），挺有特色的；马可的服务有一项细心体贴之处就是当我们办理退房手续后存包继续逛街，当大包小包拎着回到酒店取行李时，服务员很会心地把我们请到行李房最深处的一角，那里有宽大的桌椅，让你可以避免在酒店大堂中开箱、合并、装袋的尴尬举动，从容的把行李整理好再整洁的出门，真是一项贴心之举呀。', np.int64(1))
('很气派的酒店群。房间够大。唯一一点不足的地方是一个冷气口直接对着一张床！然后，酒店的早餐很少。而且，会在酒店里迷路。哈哈，因为太大了。', np.int64(1))
('酒店感觉不错，细节考虑比较周到，房价略显贵了些，标准间两个人住有点小。', np.int64(1))
('环境是很不错的,很漂亮,就是免费升级的事没有事先通知,让客人受到了不平等的待遇.意见很大.中餐不错,西餐太不地道.房间价格很便宜,套房很大,早餐也不错。', np.int64(1))
('我3.6预定好的180的标间，当我到的时候竟然说有会议房间满了，我订的房间没有了，太不讲信誉了。唉！下次再也不会住了。', np.int64(0))
('前台客房服务态度非常好！早餐很丰富，房价很干净。再接再厉！中国人的酒店就是无烟房间少，或者把烟灰缸放进抽屉就是无烟处理。我们这类75%不吸烟者需要再忍受25%烟鬼吗？希望中国办的宾馆向外企学习。请多设无烟楼层。谢谢。我原先定的房价没有无烟层楼了，我就得加付20%提升高级点的无烟层房间。Wearethe75%non-smokers,itisabouttimethattheHotelrespectustohavethefreedomoffreshair,BANtheSmokersintheCheck-inorCheck-outcounterplease.Thankyou.有或者很少房!梯不吸,是有一些吸者仍然有服!我是

In [9]:
# 创建 Dataloader
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
  texts, labels = [], []
  # batch 是一个列表，其中每个元素是你 Dataset 的 __getitem__ 返回的内容
  # batch = [ ("酒店很好", 1), ("服务很差", 0), ("一般般", 1) ]
  for item in batch:
    texts.append(item[0])
    labels.append(item[1])
  inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt") # truncation=True,超过 128 的截断
  inputs["labels"] = torch.tensor(labels)
  return inputs

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hugging Face 的 Trainer 或很多自定义模型都期望输入是一个包含 labels 键的字典

collate_func()返回的是：

{

  'input_ids': ...,

  'attention_mask': ...,

  'labels': tensor([1, 0, 1])
  
}

In [10]:
from torch.utils.data import DataLoader
# shuffle在每个 epoch 开始时，打乱 trainset 中样本的读取顺序，让模型在每个 epoch 看到不同的 batch 顺序，增强泛化能力，避免优化陷入局部模式
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [11]:
next(enumerate(validloader))[1] # 作用是从 validloader（一个 PyTorch 的 DataLoader 对象）中取出第一个 batch 的数据。

{'input_ids': tensor([[ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 6230, 2533,  ..., 8026, 8026,  102],
        [ 101, 2157, 1072,  ..., 2792, 3300,  102],
        ...,
        [ 101, 3025, 4923,  ...,    0,    0,    0],
        [ 101, 2523, 5653,  ...,    0,    0,    0],
        [ 101, 6821,  702,  ..., 5307, 1469,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
        0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 0

In [12]:
# 创建模型及优化器
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

pytorch_model.bin:   0%|          | 0.00/156M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = Adam(model.parameters(), lr=2e-5)

In [14]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [15]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1"])

In [18]:
# 训练和验证
def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        clf = evaluate()
        print(f"ep: {ep}, {clf}")

# 改进evaluate函数
def evaluate():
    model.eval()
    # acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            clf_metrics.add_batch(predictions=pred, references=batch["labels"].long())

            # acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return clf_metrics.compute()

In [19]:
!nvidia-smi

Fri Nov 28 07:45:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P0             27W /   70W |     266MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
train()

ep: 0, global_step: 0, loss: 0.7165456414222717
ep: 0, global_step: 100, loss: 0.306192547082901
ep: 0, global_step: 200, loss: 0.3037877380847931
ep: 0, {'accuracy': 0.8827319587628866, 'f1': 0.909452736318408}
ep: 1, global_step: 300, loss: 0.15528744459152222
ep: 1, global_step: 400, loss: 0.19028952717781067
ep: 1, {'accuracy': 0.8865979381443299, 'f1': 0.9135559921414538}
ep: 2, global_step: 500, loss: 0.1415272057056427
ep: 2, global_step: 600, loss: 0.22649681568145752
ep: 2, {'accuracy': 0.8994845360824743, 'f1': 0.9247104247104247}


In [ ]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [ ]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(sen)

Device set to use cuda:0


[{'label': '好评！', 'score': 0.9905791878700256}]